# 模型逻辑

## 损失函数和网络模型设计

　　这是一个分类问题，新闻总共有１４类，每条新闻数据最后需要归属到某一类新闻，虽然评价是以F１＿ｓｃｏｒｅ来进行排名，但是损失函数还是用CrossEntropyloss来计算，方便训练。  
  
　　损失函数的输入应该是一个ｌｉｎｅａｒ＋ｓｏｆｔｍａｘ的结果，而输入给ｌｉｎｅａｒ的应该是循环神经网络的最终输出的最后一个隐藏状态的结果。

## 数据集和DataLoader
　　每个训练数据包含ｔｅｘｔ和ｌａｂｅｌ两个数据。其中ｔｅｘｔ的原始数据字符串，字符串的内容为用空格隔开的语料库索引编号，现有索引编号为０－７５４９，数据中最短的ｔｅｘｔ只包含两个文字，最长的ｔｅｘｔ包含５７９２１个文字。初步思路是，先腾出索引编号‘0’给空白，然后将所有数据的text都用‘0’补齐成57921长的数组，然后将给Dataloader输出tensor，传给模型。

## 模型设计
　　模型设计为Embedding（7550，300）＋GRU（300，64）＋Linear（64，14）＋Softmax。

# 模型实现

## 数据集实现

In [31]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

class MyDataset(Dataset):
    def __init__(self,csv_path):
        csv_data = pd.read_csv(csv_path,sep='\t')    
        self.text_data = csv_data.text[:4000]
        self.label_data = csv_data.label[:4000]
        
    def __getitem__(self,index):
        #所有text内的token索引增加1，0空出来代表空格，将每个text补充成57921长
        text_str = self.text_data[index]
        text_list = [int(x)+1 for x in text_str.split()]
        text_list.extend([0]*(57921-len(text_list)))
        text_array = np.array(text_list)
        label_int = self.label_data[index]
        return text_array,label_int
        
    def __len__(self):
        return len(self.text_data)

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(7550,100)
        self.rnn = nn.GRU(100,50,batch_first=True)
        self.fc = nn.Linear(50,14)
        
    def forward(self,X):
#         print('X:',X.shape)
        X = self.embedding(X)
#         print('embedding:',X.shape)
        _,X = self.rnn(X)
#         print('rnn:',X.shape)
        X = X.squeeze(dim=0)
#         print('squeeze:',X.shape)
        y_hat = self.fc(X)
#         print('y_hat:',y_hat.shape)
        return y_hat

    
class MyTrain():
    def __init__(self,max_epoch=1,random_seed=1,lr=0.001,out_dir='./'):
        self.max_epoch = max_epoch
        self.random_seed = random_seed
        self.lr = lr
        self.out_dir = out_dir
        self.iter = 0
        
    def fix_random(self):
        import random
        import numpy as np
        import torch
        random.seed(self.random_seed)
        np.random.seed(self.random_seed)
        torch.random.manual_seed(self.random_seed)
        torch.cuda.random.manual_seed_all(self.random_seed)
        torch.backends.cudnn.deterministic = True
        print(f'random seed:{self.random_seed}')
        
    def my_train(self):
        
        max_epoch,lr = self.max_epoch,self.lr
        if self.random_seed is not None:
            self.fix_random()
        my_dataset = MyDataset('./train_set.csv')
        my_model = MyModel()
        my_model.train()          #将模型设置为训练模式
        my_optim = torch.optim.Adam(my_model.parameters(),lr=lr)
        my_loss = nn.CrossEntropyLoss()
        if torch.cuda.is_available():
            my_model.cuda()
            my_loss.cuda()
        print(f'train device:{next(iter(my_model.parameters())).device}')  #显示训练设备
        
        for epoch_index in range(max_epoch):
            loss_list = []
            f1_score_list = []
            best_f1_score = 0
            my_dataloader = DataLoader(my_dataset,batch_size=4,shuffle=True)
            my_dataloader = tqdm(my_dataloader)
            for batch_index,batch_data in enumerate(my_dataloader):
                batch_text,batch_label = batch_data
#                 print('batch_text:',batch_text.shape)
#                 print('batch_label:',batch_label)
                if torch.cuda.is_available():
                    batch_text = batch_text.cuda()
                    batch_label = batch_label.cuda()
                    
                y_hat = my_model(batch_text)
                batch_loss = my_loss(y_hat,batch_label)
                
                my_optim.zero_grad()
                batch_loss.backward()
#                 for i in my_model.parameters():
#                     print('grad:',i.grad)
                my_optim.step()
                my_optim.param_groups[0]['lr'] = lr*(0.8**((self.iter//1000)%10))
                
                #显示batch结果
                batch_lr = round(my_optim.param_groups[0]['lr'],5)
                batch_loss = round(batch_loss.item(),4)
                loss_list.append(batch_loss)
                mean_loss = round((sum(loss_list)/len(loss_list)),3)

                batch_f1_score = self.f1_score(y_hat.data,batch_label.data)
                f1_score_list.append(batch_f1_score)
                mean_f1 = round(sum(f1_score_list)/len(f1_score_list),3)
                my_dataloader.set_description(f'epoch:{epoch_index},batch:{batch_index},lr:{batch_lr},loss:{batch_loss},mean_loss:{mean_loss},mean_f1:{mean_f1}')
                
                #存储模型
                torch.save(my_model.state_dict(),os.path.join(self.out_dir,'embedding_gru_last'))                
                if batch_f1_score>best_f1_score:
                    torch.save(my_model.state_dict(),os.path.join(self.out_dir,'embedding_gru_best'))
                    best_f1_score = batch_f1_score
                    
                self.iter += 1
                
    def f1_score(self,y_hat,label,eps=1e-8):
        #y_hat(N,C),label(1)
        y_hat = y_hat.cpu()
        label = label.cpu()
        preds_list = list(torch.argmax(y_hat,dim=1).numpy())
        label_list = list(label.numpy())
        print(f'preds:{preds_list},label:{label_list}')
        class_index_list = []
        for class_index in label_list:
            if class_index not in class_index_list:
                class_index_list.append(class_index)

        f1_score_list = []
        for index in class_index_list:
            if index not in preds_list:
                sub_f1_score = 0
            else:
                tp = 0
                fp = 0
                fn = 0
                for i in range(len(preds_list)):
                    if preds_list[i] == index and label_list[i] == index:
                        tp+=1
                    if preds_list[i] == index and label_list[i] != index: 
                        fp+=1
                    if preds_list[i] != index and label_list[i] == index:
                        fn+=1
                prec_val = tp/(tp+fp) 
                recall_val = tp/(tp+fn)
                sub_f1_score = 2*(prec_val*recall_val)/(prec_val+recall_val+eps)
            f1_score_list.append(sub_f1_score)

        batch_f1_score = sum(f1_score_list)/len(f1_score_list)

        return batch_f1_score
        
        
        
#     #验证所有的验证集数据
#     def my_valid(self,valid_dataloader,model,criterion):

#         #将模型设置为计算模型
#         model.eval()

#         total_loss = 0
#         with torch.no_grad():
#             for batch_index,valid_data in enumerate(valid_dataloader):
#                 batch_feat,batch_label = valid_data
#                 y_hat = model(batch_feat)
#                 loss = criterion(y_hat,batch_label)
#                 total_loss+=loss
#             total_loss/=(batch_index+1)

#         return total_loss

In [32]:
if __name__ == '__main__':
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    MyTrain(max_epoch=1,random_seed=None).my_train()

train device:cuda:0


epoch:0,batch:0,lr:0.001,loss:2.9985,mean_loss:2.998,mean_f1:0.0:   0%|               | 1/1000 [00:00<06:37,  2.51it/s]

preds:[3, 3, 3, 3],label:[0, 13, 1, 11]


epoch:0,batch:1,lr:0.001,loss:2.6335,mean_loss:2.816,mean_f1:0.05:   0%|              | 2/1000 [00:00<05:30,  3.02it/s]

preds:[3, 3, 3, 3],label:[7, 5, 3, 0]


...

preds:[1, 1, 1, 1],label:[1, 0, 1, 2]


epoch:0,batch:998,lr:0.001,loss:1.9501,mean_loss:2.294,mean_f1:0.091: 100%|████████▉| 999/1000 [04:17<00:00,  3.89it/s]

preds:[1, 1, 1, 1],label:[0, 3, 2, 0]


epoch:0,batch:999,lr:0.001,loss:2.2416,mean_loss:2.294,mean_f1:0.091: 100%|████████| 1000/1000 [04:17<00:00,  3.88it/s]

preds:[1, 1, 1, 1],label:[1, 8, 4, 0]


　　训练不动，f1分数和损失都没有变化。初步判断是两个原因，一是文本数据的T太大，每个文本有59000多的长度，意味着一个GRU内部要进行接近6万次的传递才能输出最终的隐藏状态，在反向传播时，梯度越往前则改变越小，导致一个文本数据的前面一些文字对模型的影响越来越小，数据失去效果。二是数据的分类不均衡，导致模型只要预测为0、1、2、3就能获得相对来说还可以的结果，不需要进行学习了。  
　　对应的解决思路有三个，第一个问题有两个思路，思路一，先进行几次卷积，把Ｔ降下来，然后再给ｒｎｎ，思路二，对ｒｎｎ按照一定Ｔ距离截断反向传播，避免反向传播的梯度一直叠加，第二个问题，为ＣｒｏｓｓＥｎｔｒｏｐｙＬｏｓｓ加上权重，权重由数据集标签算得。  
　　先尝试第一个，在ｒｎｎ之前加卷积，把Ｔ给降下来。

In [1]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

class MyDataset(Dataset):
    def __init__(self,csv_path):
        csv_data = pd.read_csv(csv_path,sep='\t')    
        self.text_data = csv_data.text[:4000]
        self.label_data = csv_data.label[:4000]
        
    def __getitem__(self,index):
        #所有text内的token索引增加1，0空出来代表空格，将每个text补充成57921长
        text_str = self.text_data[index]
        text_list = [int(x)+1 for x in text_str.split()]
        text_list.extend([0]*(57921-len(text_list)))
        text_array = np.array(text_list)
        label_int = self.label_data[index]
        return text_array,label_int
        
    def __len__(self):
        return len(self.text_data)

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(7550,100)
        self.cnn = nn.Sequential(nn.Conv1d(57921,580,kernel_size=1),
                                 nn.Conv1d(580,58,kernel_size=1))
        
        self.rnn = nn.GRU(100,50,batch_first=True)
        self.fc = nn.Linear(50,14)
        
    def forward(self,X):
#         print('X:',X.shape)
        X = self.embedding(X)
#         print('embedding:',X.shape)
        X = self.cnn(X)
#         print('cnn:',X.shape)
        _,X = self.rnn(X)
#         print('rnn:',X.shape)
        X = X.squeeze(dim=0)
#         print('squeeze:',X.shape)
        y_hat = self.fc(X)
#         print('y_hat:',y_hat.shape)
        return y_hat

    
class MyTrain():
    def __init__(self,max_epoch=1,random_seed=1,lr=0.001,out_dir='./'):
        self.max_epoch = max_epoch
        self.random_seed = random_seed
        self.lr = lr
        self.out_dir = out_dir
        self.iter = 0
        
    def fix_random(self):
        import random
        import numpy as np
        import torch
        random.seed(self.random_seed)
        np.random.seed(self.random_seed)
        torch.random.manual_seed(self.random_seed)
        torch.cuda.random.manual_seed_all(self.random_seed)
        torch.backends.cudnn.deterministic = True
        print(f'random seed:{self.random_seed}')
        
    def my_train(self):
        
        max_epoch,lr = self.max_epoch,self.lr
        if self.random_seed is not None:
            self.fix_random()
        my_dataset = MyDataset('./train_set.csv')
        my_model = MyModel()
        my_model.train()          #将模型设置为训练模式
        my_optim = torch.optim.Adam(my_model.parameters(),lr=lr)
        my_loss = nn.CrossEntropyLoss()
        if torch.cuda.is_available():
            my_model.cuda()
            my_loss.cuda()
        print(f'train device:{next(iter(my_model.parameters())).device}')  #显示训练设备
        
        for epoch_index in range(max_epoch):
            loss_list = []
            f1_score_list = []
            best_f1_score = 0
            my_dataloader = DataLoader(my_dataset,batch_size=5,shuffle=True)
            my_dataloader = tqdm(my_dataloader)
            for batch_index,batch_data in enumerate(my_dataloader):
                batch_text,batch_label = batch_data
#                 print('batch_text:',batch_text.shape)
#                 print('batch_label:',batch_label)
                if torch.cuda.is_available():
                    batch_text = batch_text.cuda()
                    batch_label = batch_label.cuda()
                    
                y_hat = my_model(batch_text)
                batch_loss = my_loss(y_hat,batch_label)
                
                my_optim.zero_grad()
                batch_loss.backward()
#                 for i in my_model.parameters():
#                     print('grad:',i.grad)
                my_optim.step()
                my_optim.param_groups[0]['lr'] = lr*(0.8**((self.iter//1000)%10))
                
                #显示batch结果
                batch_lr = round(my_optim.param_groups[0]['lr'],5)
                batch_loss = round(batch_loss.item(),4)
                loss_list.append(batch_loss)
                mean_loss = round((sum(loss_list)/len(loss_list)),3)

                batch_f1_score = self.f1_score(y_hat.data,batch_label.data)
                f1_score_list.append(batch_f1_score)
                mean_f1 = round(sum(f1_score_list)/len(f1_score_list),3)
                my_dataloader.set_description(f'epoch:{epoch_index},batch:{batch_index},lr:{batch_lr},loss:{batch_loss},mean_loss:{mean_loss},mean_f1:{mean_f1}')
                
                #存储模型
                torch.save(my_model.state_dict(),os.path.join(self.out_dir,'embedding_gru_last'))                
                if batch_f1_score>best_f1_score:
                    torch.save(my_model.state_dict(),os.path.join(self.out_dir,'embedding_gru_best'))
                    best_f1_score = batch_f1_score
                    
                self.iter += 1
                
    def f1_score(self,y_hat,label,eps=1e-8):
        #y_hat(N,C),label(1)
        y_hat = y_hat.cpu()
        label = label.cpu()
        preds_list = list(torch.argmax(y_hat,dim=1).numpy())
        label_list = list(label.numpy())
        print(f'preds:{preds_list},label:{label_list}')
        class_index_list = []
        for class_index in label_list:
            if class_index not in class_index_list:
                class_index_list.append(class_index)

        f1_score_list = []
        for index in class_index_list:
            if index not in preds_list:
                sub_f1_score = 0
            else:
                tp = 0
                fp = 0
                fn = 0
                for i in range(len(preds_list)):
                    if preds_list[i] == index and label_list[i] == index:
                        tp+=1
                    if preds_list[i] == index and label_list[i] != index: 
                        fp+=1
                    if preds_list[i] != index and label_list[i] == index:
                        fn+=1
                prec_val = tp/(tp+fp) 
                recall_val = tp/(tp+fn)
                sub_f1_score = 2*(prec_val*recall_val)/(prec_val+recall_val+eps)
            f1_score_list.append(sub_f1_score)

        batch_f1_score = sum(f1_score_list)/len(f1_score_list)

        return batch_f1_score


In [2]:
if __name__ == '__main__':
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    MyTrain(max_epoch=1,random_seed=None).my_train()

train device:cuda:0


epoch:0,batch:0,lr:0.001,loss:2.6065,mean_loss:2.607,mean_f1:0.143:   0%|                      | 0/800 [00:01<?, ?it/s]

preds:[1, 1, 1, 1, 1],label:[4, 1, 8, 0, 1]


epoch:0,batch:1,lr:0.001,loss:2.8546,mean_loss:2.731,mean_f1:0.071:   0%|              | 1/800 [00:04<55:12,  4.15s/it]

preds:[3, 3, 3, 3, 3],label:[9, 2, 0, 0, 0]


...

preds:[0, 0, 0, 0, 0],label:[9, 0, 0, 0, 10]


epoch:0,batch:157,lr:0.001,loss:1.8793,mean_loss:2.369,mean_f1:0.069:  20%|█▉        | 157/800 [02:45<12:38,  1.18s/it]

preds:[0, 0, 0, 0, 0],label:[0, 7, 0, 0, 0]


epoch:0,batch:157,lr:0.001,loss:1.8793,mean_loss:2.369,mean_f1:0.069:  20%|█▉        | 157/800 [02:46<11:22,  1.06s/it]


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\caffe2\serialize\inline_container.cc:319] . unexpected pos 3021440 vs 3021360

　　直接进行卷积对减少模型复杂度没有用，反而因为增加了卷积，让模型变得更大，更难以训练了。基于这样的情况，又回过头分析了一下，text的长度分布问题，发现绝大多数的text的长度都是在5000以下，所以前面思路中把每个text都扩充到59000多可能有些浪费，试试看把每个text控制在5000长度再试试。

In [1]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

class MyDataset(Dataset):
    def __init__(self,csv_path):
        csv_data = pd.read_csv(csv_path,sep='\t')    
        self.text_data = csv_data.text
        self.label_data = csv_data.label
        
    def __getitem__(self,index):
        #所有text内的token索引增加1，0空出来代表空格，将每个text控制在1000长
        text_str = self.text_data[index]
        text_list = [int(x)+1 for x in text_str.split()]
        if len(text_list)>=1000:
            text_list = text_list[:1000]
        else:
            text_list.extend([0]*(1000-len(text_list)))
        text_array = np.array(text_list)
        label_array = np.array(self.label_data[index])
        return text_array,label_array
        
    def __len__(self):
        return len(self.text_data)

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(7550,100)
        self.rnn = nn.GRU(100,50,batch_first=True)
        self.fc = nn.Linear(50,14)
        
    def forward(self,X):
#         print('X:',X.shape)
        X = self.embedding(X)
#         print('embedding:',X.shape)
        _,X = self.rnn(X)
#         print('rnn:',X.shape)
        X = X.squeeze(dim=0)
#         print('squeeze:',X.shape)
        y_hat = self.fc(X)
#         print('y_hat:',y_hat.shape)
        return y_hat

    
class MyTrain():
    def __init__(self,max_epoch=1,random_seed=1,lr=0.001,out_dir='./'):
        self.max_epoch = max_epoch
        self.random_seed = random_seed
        self.lr = lr
        self.out_dir = out_dir
        self.iter = 0
        
    def fix_random(self):
        import random
        import numpy as np
        import torch
        random.seed(self.random_seed)
        np.random.seed(self.random_seed)
        torch.random.manual_seed(self.random_seed)
        torch.cuda.random.manual_seed_all(self.random_seed)
        torch.backends.cudnn.deterministic = True
        print(f'random seed:{self.random_seed}')
        
    def my_train(self):
        
        max_epoch,lr = self.max_epoch,self.lr
        if self.random_seed is not None:
            self.fix_random()
        my_dataset = MyDataset('./train_set.csv')
        my_model = MyModel()
        my_model.train()          #将模型设置为训练模式
        my_optim = torch.optim.Adam(my_model.parameters(),lr=lr)
        my_loss = nn.CrossEntropyLoss()
        if torch.cuda.is_available():
            my_model.cuda()
            my_loss.cuda()
        print(f'train device:{next(iter(my_model.parameters())).device}')  #显示训练设备
        
        for epoch_index in range(max_epoch):
            loss_list = []
            f1_score_list = []
            best_f1_score = 0
            my_dataloader = DataLoader(my_dataset,batch_size=20,shuffle=True)
            my_dataloader = tqdm(my_dataloader)
            for batch_index,batch_data in enumerate(my_dataloader):
                batch_text,batch_label = batch_data
                print('batch_text:',batch_text.shape)
                print('batch_label:',batch_label)
                if torch.cuda.is_available():
                    batch_text = batch_text.cuda()
                    batch_label = batch_label.cuda()
                    
                y_hat = my_model(batch_text)
                batch_loss = my_loss(y_hat,batch_label)
                
                my_optim.zero_grad()
                batch_loss.backward()
#                 for i in my_model.parameters():
#                     print('grad:',i.grad)
                my_optim.step()
                my_optim.param_groups[0]['lr'] = lr*(0.8**((self.iter//1000)%10))
                
                #显示batch结果
                batch_lr = round(my_optim.param_groups[0]['lr'],5)
                batch_loss = round(batch_loss.item(),4)
                loss_list.append(batch_loss)
                mean_loss = round((sum(loss_list)/len(loss_list)),3)

                batch_f1_score = self.f1_score(y_hat.data,batch_label.data)
                f1_score_list.append(batch_f1_score)
                mean_f1 = round(sum(f1_score_list)/len(f1_score_list),3)
                my_dataloader.set_description(f'epoch:{epoch_index},batch:{batch_index},lr:{batch_lr},loss:{batch_loss},mean_loss:{mean_loss},mean_f1:{mean_f1}')
                
                #存储模型
                torch.save(my_model.state_dict(),os.path.join(self.out_dir,'embedding_gru_last'))                
                if batch_f1_score>best_f1_score:
                    torch.save(my_model.state_dict(),os.path.join(self.out_dir,'embedding_gru_best'))
                    best_f1_score = batch_f1_score
                    
                self.iter += 1
                
    def f1_score(self,y_hat,label,eps=1e-8):
        #y_hat(N,C),label(1)
        y_hat = y_hat.cpu()
        label = label.cpu()
        preds_list = list(torch.argmax(y_hat,dim=1).numpy())
        label_list = list(label.numpy())
#         print(f'preds:{preds_list},label:{label_list}')
        class_index_list = []
        for class_index in label_list:
            if class_index not in class_index_list:
                class_index_list.append(class_index)

        f1_score_list = []
        for index in class_index_list:
            if index not in preds_list:
                sub_f1_score = 0
            else:
                tp = 0
                fp = 0
                fn = 0
                for i in range(len(preds_list)):
                    if preds_list[i] == index and label_list[i] == index:
                        tp+=1
                    if preds_list[i] == index and label_list[i] != index: 
                        fp+=1
                    if preds_list[i] != index and label_list[i] == index:
                        fn+=1
                prec_val = tp/(tp+fp) 
                recall_val = tp/(tp+fn)
                sub_f1_score = 2*(prec_val*recall_val)/(prec_val+recall_val+eps)
            f1_score_list.append(sub_f1_score)

        batch_f1_score = sum(f1_score_list)/len(f1_score_list)

        return batch_f1_score
        
        
        
#     #验证所有的验证集数据
#     def my_valid(self,valid_dataloader,model,criterion):

#         #将模型设置为计算模型
#         model.eval()

#         total_loss = 0
#         with torch.no_grad():
#             for batch_index,valid_data in enumerate(valid_dataloader):
#                 batch_feat,batch_label = valid_data
#                 y_hat = model(batch_feat)
#                 loss = criterion(y_hat,batch_label)
#                 total_loss+=loss
#             total_loss/=(batch_index+1)

#         return total_loss

In [2]:
if __name__ == '__main__':
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    MyTrain(max_epoch=10,random_seed=None).my_train()

train device:cuda:0


epoch:0,batch:0,lr:0.001,loss:2.7137,mean_loss:2.714,mean_f1:0.0:   0%|                      | 0/10000 [00:00<?, ?it/s]

batch_text: torch.Size([20, 1000])
batch_label: tensor([ 6,  1,  3,  0,  6,  2,  1,  8, 13,  1,  1,  1,  1,  4,  2,  8,  7,  9,
         0,  7])


epoch:0,batch:1,lr:0.001,loss:2.7193,mean_loss:2.716,mean_f1:0.0:   0%|              | 1/10000 [00:00<30:05,  5.54it/s]

batch_text: torch.Size([20, 1000])
batch_label: tensor([7, 2, 4, 2, 0, 0, 2, 6, 0, 8, 1, 1, 1, 2, 1, 0, 8, 2, 1, 0])


...

epoch:0,batch:776,lr:0.001,loss:2.1546,mean_loss:2.267,mean_f1:0.079:   8%|▌       | 776/10000 [00:19<03:48, 40.34it/s]

batch_text: torch.Size([20, 1000])
batch_label: tensor([8, 3, 1, 7, 3, 1, 4, 5, 3, 5, 1, 3, 0, 8, 4, 7, 1, 0, 7, 2])
batch_text: torch.Size([20, 1000])
batch_label: tensor([ 0,  0,  6,  1,  6,  2,  2,  0,  0,  2,  0,  1,  2,  1,  5,  8,  0,  0,
        12, 10])
batch_text: torch.Size([20, 1000])
batch_label: tensor([ 5, 10,  0,  5,  0,  7,  0,  5, 10,  7,  3,  1, 13,  2,  8,  2,  3,  3,
         0,  5])
batch_text: torch.Size([20, 1000])
batch_label: tensor([5, 2, 1, 1, 2, 4, 4, 0, 0, 1, 6, 7, 7, 1, 1, 2, 4, 6, 3, 8])
batch_text: torch.Size([20, 1000])
batch_label: tensor([ 2,  3,  0,  0,  6, 11,  9,  3,  2,  1,  0,  4,  0,  1,  4,  3,  0,  5,
         1,  0])
batch_text: torch.Size([20, 1000])
batch_label: tensor([7, 9, 6, 0, 2, 0, 0, 0, 5, 8, 7, 1, 1, 2, 1, 7, 0, 2, 2, 0])
batch_text: torch.Size([20, 1000])
batch_label: tensor([7, 0, 0, 9, 2, 4, 1, 2, 1, 1, 5, 5, 1, 1, 0, 5, 2, 9, 2, 9])
batch_text: torch.Size([20, 1000])
batch_label: tensor([ 0,  2,  0,  7,  1,  2, 13,  3,  2,  0,  

epoch:0,batch:781,lr:0.001,loss:2.4512,mean_loss:2.268,mean_f1:0.079:   8%|▋       | 782/10000 [00:19<03:52, 39.59it/s]

batch_text: torch.Size([20, 1000])
batch_label: tensor([ 0, 10,  2,  1,  2,  0,  7,  0,  8,  9,  3,  3,  7, 13,  1,  1,  8,  2,
         2,  1])
batch_text: torch.Size([20, 1000])
batch_label: tensor([ 0,  1, 11,  9,  0,  4,  4,  3,  6,  0, 10,  1,  4, 10,  3,  1,  3, 10,
         5,  2])
batch_text: torch.Size([20, 1000])
batch_label: tensor([ 0,  0,  8,  1, 11,  2,  1,  1,  0,  0, 11,  2,  0,  0,  1,  0,  1,  3,
         3,  3])
batch_text: torch.Size([20, 1000])
batch_label: tensor([11,  5,  4,  3,  2,  3,  6,  2,  1,  2,  0,  3,  3,  0,  2,  1,  1,  1,
         0,  5])
batch_text: torch.Size([20, 1000])
batch_label: tensor([8, 8, 8, 4, 4, 0, 4, 4, 3, 4, 3, 4, 5, 8, 7, 1, 1, 1, 4, 3])
batch_text: torch.Size([20, 1000])
batch_label: tensor([5, 4, 3, 1, 0, 1, 6, 6, 2, 1, 1, 3, 3, 2, 5, 2, 0, 0, 2, 0])


RuntimeError: unique_by_key: failed on 2nd step: cudaErrorAssert: device-side assert triggered

　　上面报了个错，查了之后发现网上说很有可能是使用ｃｒｏｓｓＥｎｔｒｏｐｙＬｏｓｓ的时候，输入数据的类别数目跟实际的类别数目不对照。首先检查了一下ｌａｂｅｌ的最大最小值以及每个数据的ｌａｂｅｌ的长度，没有发现问题。后来有一个说法是说可能跟稀疏矩阵有关系，突然意识到，之前ｔｏｋｅｎ的最大编号是７５４９，但是我把每个ｔｏｋｅｎ增加了１，然后把０空出来作为新增的空格，但是在ｅｍｂｅｄｄｉｎｇ层中，我设置的形状是７５５０，７５５０指的应该是ｔｏｋｅｎ的最大范围，在我增加０之后，这个范围从０－７５４９变成了０－７５５０，现在ｅｍｂｅｄｄｉｎｇ层的形状应该是７５５１才能正常使用，把这个改过来之后再尝试。

In [5]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

class MyDataset(Dataset):
    def __init__(self,csv_path):
        csv_data = pd.read_csv(csv_path,sep='\t')    
        self.text_data = csv_data.text
        self.label_data = csv_data.label
        
    def __getitem__(self,index):
        #所有text内的token索引增加1，0空出来代表空格，将每个text控制在1000长
        text_str = self.text_data[index]
        text_list = [int(x)+1 for x in text_str.split()]
        if len(text_list)>=1000:
            text_list = text_list[:1000]
        else:
            text_list.extend([0]*(1000-len(text_list)))
        text_array = np.array(text_list)
        label_array = np.array(self.label_data[index])
        return text_array,label_array
        
    def __len__(self):
        return len(self.text_data)

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(7551,100)
        self.rnn = nn.GRU(100,50,batch_first=True)
        self.fc = nn.Linear(50,14)
        
    def forward(self,X):
#         print('X:',X.shape)
        X = self.embedding(X)
#         print('embedding:',X.shape)
        _,X = self.rnn(X)
#         print('rnn:',X.shape)
        X = X.squeeze(dim=0)
#         print('squeeze:',X.shape)
        y_hat = self.fc(X)
#         print('y_hat:',y_hat.shape)
        return y_hat

    
class MyTrain():
    def __init__(self,max_epoch=1,random_seed=1,lr=0.001,out_dir='./'):
        self.max_epoch = max_epoch
        self.random_seed = random_seed
        self.lr = lr
        self.out_dir = out_dir
        self.iter = 0
        
    def fix_random(self):
        import random
        import numpy as np
        import torch
        random.seed(self.random_seed)
        np.random.seed(self.random_seed)
        torch.random.manual_seed(self.random_seed)
        torch.cuda.random.manual_seed_all(self.random_seed)
        torch.backends.cudnn.deterministic = True
        print(f'random seed:{self.random_seed}')
        
    def my_train(self):
        
        max_epoch,lr = self.max_epoch,self.lr
        if self.random_seed is not None:
            self.fix_random()
        my_dataset = MyDataset('./train_set.csv')
        my_model = MyModel()
        my_model.train()          #将模型设置为训练模式
        my_optim = torch.optim.Adam(my_model.parameters(),lr=lr)
        my_loss = nn.CrossEntropyLoss()
        if torch.cuda.is_available():
            my_model.cuda()
            my_loss.cuda()
        print(f'train device:{next(iter(my_model.parameters())).device}')  #显示训练设备
        
        for epoch_index in range(max_epoch):
            loss_list = []
            f1_score_list = []
            best_f1_score = 0
            my_dataloader = DataLoader(my_dataset,batch_size=100,shuffle=True)
            my_dataloader = tqdm(my_dataloader)
            for batch_index,batch_data in enumerate(my_dataloader):
                batch_text,batch_label = batch_data
#                 print('batch_text:',batch_text.shape)
#                 print('batch_label:',batch_label)
                if torch.cuda.is_available():
                    batch_text = batch_text.cuda()
                    batch_label = batch_label.cuda()
                    
                y_hat = my_model(batch_text)
                batch_loss = my_loss(y_hat,batch_label)
                
                my_optim.zero_grad()
                batch_loss.backward()
#                 for i in my_model.parameters():
#                     print('grad:',i.grad)
                my_optim.step()
                my_optim.param_groups[0]['lr'] = lr*(0.8**(epoch_index%10))
                
                #显示batch结果
                batch_lr = round(my_optim.param_groups[0]['lr'],5)
                batch_loss = round(batch_loss.item(),4)
                loss_list.append(batch_loss)
                mean_loss = round((sum(loss_list)/len(loss_list)),3)

                batch_f1_score = self.f1_score(y_hat.data,batch_label.data)
                f1_score_list.append(batch_f1_score)
                mean_f1 = round(sum(f1_score_list)/len(f1_score_list),3)
                my_dataloader.set_description(f'epoch:{epoch_index},batch:{batch_index},lr:{batch_lr},loss:{batch_loss},mean_loss:{mean_loss},mean_f1:{mean_f1}')
                
                #存储模型
                torch.save(my_model.state_dict(),os.path.join(self.out_dir,'embedding_gru_last'))                
                if batch_f1_score>best_f1_score:
                    torch.save(my_model.state_dict(),os.path.join(self.out_dir,'embedding_gru_best'))
                    best_f1_score = batch_f1_score
                    
                self.iter += 1
                
    def f1_score(self,y_hat,label,eps=1e-8):
        #y_hat(N,C),label(1)
        y_hat = y_hat.cpu()
        label = label.cpu()
        preds_list = list(torch.argmax(y_hat,dim=1).numpy())
        label_list = list(label.numpy())
#         print(f'preds:{preds_list},label:{label_list}')
        class_index_list = []
        for class_index in label_list:
            if class_index not in class_index_list:
                class_index_list.append(class_index)

        f1_score_list = []
        for index in class_index_list:
            if index not in preds_list:
                sub_f1_score = 0
            else:
                tp = 0
                fp = 0
                fn = 0
                for i in range(len(preds_list)):
                    if preds_list[i] == index and label_list[i] == index:
                        tp+=1
                    if preds_list[i] == index and label_list[i] != index: 
                        fp+=1
                    if preds_list[i] != index and label_list[i] == index:
                        fn+=1
                prec_val = tp/(tp+fp) 
                recall_val = tp/(tp+fn)
                sub_f1_score = 2*(prec_val*recall_val)/(prec_val+recall_val+eps)
            f1_score_list.append(sub_f1_score)

        batch_f1_score = sum(f1_score_list)/len(f1_score_list)

        return batch_f1_score
        
        
        
#     #验证所有的验证集数据
#     def my_valid(self,valid_dataloader,model,criterion):

#         #将模型设置为计算模型
#         model.eval()

#         total_loss = 0
#         with torch.no_grad():
#             for batch_index,valid_data in enumerate(valid_dataloader):
#                 batch_feat,batch_label = valid_data
#                 y_hat = model(batch_feat)
#                 loss = criterion(y_hat,batch_label)
#                 total_loss+=loss
#             total_loss/=(batch_index+1)

#         return total_loss

In [8]:
if __name__ == '__main__':
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    MyTrain(max_epoch=100,random_seed=1).my_train()

random seed:1
train device:cuda:0


epoch:0,batch:1999,lr:0.001,loss:0.3871,mean_loss:1.061,mean_f1:0.463: 100%|███████| 2000/2000 [02:01<00:00, 16.47it/s]
epoch:1,batch:1999,lr:0.0008,loss:0.2512,mean_loss:0.306,mean_f1:0.851: 100%|██████| 2000/2000 [02:01<00:00, 16.49it/s]
epoch:2,batch:1999,lr:0.00064,loss:0.1788,mean_loss:0.229,mean_f1:0.899: 100%|█████| 2000/2000 [02:00<00:00, 16.54it/s]
epoch:3,batch:1999,lr:0.00051,loss:0.0831,mean_loss:0.195,mean_f1:0.916: 100%|█████| 2000/2000 [02:01<00:00, 16.49it/s]
epoch:4,batch:1999,lr:0.00041,loss:0.1926,mean_loss:0.172,mean_f1:0.927: 100%|█████| 2000/2000 [02:01<00:00, 16.48it/s]
epoch:5,batch:1999,lr:0.00033,loss:0.1434,mean_loss:0.157,mean_f1:0.935: 100%|█████| 2000/2000 [02:01<00:00, 16.48it/s]
epoch:6,batch:1999,lr:0.00026,loss:0.1661,mean_loss:0.145,mean_f1:0.94: 100%|██████| 2000/2000 [02:01<00:00, 16.49it/s]
epoch:7,batch:1999,lr:0.00021,loss:0.1038,mean_loss:0.136,mean_f1:0.944: 100%|█████| 2000/2000 [02:01<00:00, 16.46it/s]
epoch:8,batch:1999,lr:0.00017,loss:0.147